[View in Colaboratory](https://colab.research.google.com/github/iegorval/neural_nets/blob/master/Simulated.ipynb)

# Getting Ready
Let's first import the required libraries.

In [0]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras.layers import LSTM, Input, Activation, Dense, TimeDistributed
from keras.models import Sequential

Now import the data and display the information about it.

In [0]:
data_henon = pd.read_csv("henon.csv", header=None)
data_lorenz = pd.read_csv("lorenz.csv", header=None)
data_rossler = pd.read_csv("rossler.csv", header=None)
data_henon.describe()

In [0]:
data_lorenz.describe()

In [0]:
data_rossler.describe()

In [0]:
data_lorenz_cut = pd.concat([pd.DataFrame([data_lorenz[0][i*10]], columns=['0']) for i in range(len(data_lorenz[0])//10)])

**Choose the dataset used for the learning**

In [0]:
train_data = data_lorenz
plt.plot(range(200), train_data[:200])

We have to pre-process the input, so it is not a single number, but a sequence of the chosen length (context_length).

In [0]:
data_np = np.array(train_data)
Tx, n_values = data_np.shape
train_np = train_data.values

context_length = 50
y_train = train_np[context_length:,:]
train_np_expanded = np.repeat(train_np[:,np.newaxis], context_length, 1)
for i in range(1,context_length):
    train_np_expanded[:-i,i,:] = train_np_expanded[i:,i,:]
x_train = train_np_expanded[:-context_length,:,:]

print(x_train.shape)
print(y_train.shape)

# Model
Prepare the RNN model in Keras.

In [0]:
def get_model(c):
    model = Sequential()
    model.add(LSTM(64, input_shape=(c,1), return_sequences=False))
    model.add(Dense(1))
    model.add(Activation("linear"))
    return model

# Experiment

In [0]:
model = get_model(context_length)
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mse']) 

In [0]:
model.summary()

In [0]:
history = model.fit(x_train, y_train, epochs=40)

In [0]:
plt.plot(range(20), history.history['mean_squared_error'][:20], 'r')
plt.ylabel("Mean Squared Error")
plt.xlabel("Epochs")

In [0]:
model.evaluate(x=x_train, y=y_train)

# Results
Get the signal by giving the real-data sequence and predicting the next value.

In [0]:
Ty = train_np.shape[0]
y_pred = list()
for y in range(Ty):
    example = np.reshape(train_np_expanded[y,:,:], (1,context_length,1))
    y_pred.append(model.predict(example).tolist()[0][0])

In [0]:
plt.subplot(2,1,1)

plt.plot(y_pred[:200],'r')
plt.ylabel("predicted signal")
plt.subplot(2,1,2)
plt.plot(train_np[context_length:200+context_length])
plt.ylabel("henon time series")

In [0]:
plt.plot(range(200),train_np[context_length:200+context_length],'b',range(200),y_pred[:200],'r')
plt.legend(("ground truth","predicted signal"))

Free-run test of the model.

In [0]:
Ty = train_np.shape[0]
y_gen = list()
example = np.reshape(train_np[:context_length], (1,context_length,1))
for y in range(Ty):
    prediction = model.predict(example).tolist()[0][0]
    for i in range(context_length-1):
        example[0][i] = example[0][i+1]
    example[0][context_length-1] = prediction
    y_gen.append(prediction)

In [0]:
plt.plot(y_gen[:200],'r')
plt.ylabel("generated signal")